In [1]:
#### Imports para entrenamiento, predicción y evaluación del modelo ####
#### NO QUITAR ####
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
#### Código Agregado ####
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')
### Loading Data


In [2]:
fulldata_path = 'tumama.csv'
fulldata = pd.read_csv(fulldata_path)

---
# Comienzo de encoding variables categóricas
---

In [3]:
num_cols = fulldata._get_numeric_data().columns 
cat_columns = list(set(fulldata.columns) - set(num_cols))

In [4]:
cat_columns

['DEVICE_VENDOR_NAME',
 'PREP_RECH_LAST_DAY_12W',
 'Date',
 'DEVICE_MODEL_NAME',
 'Period',
 'MICROSEGMENTATION',
 'SEGMENTATION',
 'NETWORK_TECH',
 'PREP_RECH_CHNNL_MODE12W',
 'VALUE_SEGMENT',
 'COMMERCIAL_ACTIVATION_DATE',
 'SUSCRIBER_KEY']

In [5]:
fulldata[cat_columns].nunique()

DEVICE_VENDOR_NAME               211
PREP_RECH_LAST_DAY_12W            84
Date                            1417
DEVICE_MODEL_NAME               2141
Period                            47
MICROSEGMENTATION                 25
SEGMENTATION                       7
NETWORK_TECH                       3
PREP_RECH_CHNNL_MODE12W         5677
VALUE_SEGMENT                      4
COMMERCIAL_ACTIVATION_DATE      1417
SUSCRIBER_KEY                 226206
dtype: int64

### Después de inspeccionar estas variables, con imágenes de value counts, nos detenemos en el caso de DEVICE_VENDOR_NAME

In [25]:
fulldata.DEVICE_VENDOR_NAME.value_counts(normalize=True)[0:10]

SAMSUNG       0.640447
HUAWEI        0.173939
LG            0.050578
BLU           0.032395
NOKIA         0.022347
APPLE         0.016551
UNKNOWN       0.011083
GREYMARKET    0.007873
HYUNDAI       0.006963
ALCATEL       0.005866
Name: DEVICE_VENDOR_NAME, dtype: float64

### Vemos que la mayoria se concentra en 6 marcas, remplazaremos con 'Other' para poder aplicar hot encoding

In [26]:
n = 6
most_frequent_vendors = fulldata['DEVICE_VENDOR_NAME'].value_counts()[:n].index.tolist()
fulldata.loc[~fulldata["DEVICE_VENDOR_NAME"].isin(most_frequent_vendors), "DEVICE_VENDOR_NAME"] = "Other"

In [29]:
fulldata.DEVICE_VENDOR_NAME.value_counts(normalize=1)

SAMSUNG    0.640447
HUAWEI     0.173939
Other      0.063743
LG         0.050578
BLU        0.032395
NOKIA      0.022347
APPLE      0.016551
Name: DEVICE_VENDOR_NAME, dtype: float64

### Aplicaremos one hot encode a las columnas SEGMENTATION, VALUE_SEGMENT, NETWORK_TECH, DEVICE_VENDOR_NAME

In [30]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [10]:
fulldata.shape

(226206, 91)

In [32]:
columns_to_encode = ['SEGMENTATION', 'VALUE_SEGMENT', 'NETWORK_TECH', 'DEVICE_VENDOR_NAME']
for column in columns_to_encode: 
    fulldata = encode_and_bind(fulldata, column)
fulldata.shape

(226206, 126)

---
# Correlación variables con target
---

In [45]:
feature_target_correlation = fulldata[fulldata.columns[1:]].corr()['Target'].dropna()
feature_targe_correlation.abs().sort_values(ascending=False)[0:15]

Target                          1.000000
PREP_RECH_NDAYS_LASTRECH_12W    0.551611
SEGMENTATION_7-LOW              0.484010
SEGMENTATION_7-LOW              0.484010
SEGMENTATION_7-LOW              0.484010
SEGMENTATION_7-LOW              0.484010
VALUE_SEGMENT_4 - BRONZE        0.413995
VALUE_SEGMENT_4 - BRONZE        0.413995
VALUE_SEGMENT_4 - BRONZE        0.413995
VALUE_SEGMENT_4 - BRONZE        0.413995
PACK_DATA_EXP_W1                0.396922
PACK_DATA_EXP_W2                0.396833
PACK_DATA_AMT_W2                0.372390
PACK_DATA_EXP_W3                0.371835
PACK_DATA_AMT_W1                0.363064
Name: Target, dtype: float64

## Consultar!